# Differential gene expression analysis per cell-type between conditions

In [6]:
import pandas as pd
import numpy as np
import os
import subprocess as sp
import decoupler as dc
import scanpy as sc


In [7]:
#Load data
input_adata = "/data/projects/2023/atlas_protocol/input_data_zenodo/atlas-integrated-annotated.h5ad"
adata = sc.read_h5ad(input_adata)

In [8]:
#Processing
# Gene symbols required 
#adata.var.head()
# All cells are annotated 
#adata.obs["cell_type"].isnull().value_counts()
#adata.obs["cell_type_coarse"].unique()

In [9]:
adata = adata[adata.obs["origin"].isin(["tumor_primary"])]
adata = adata[adata.obs["condition"].isin(["LUAD", "LUSC"])]

In [31]:
cell_type[0]

'B cell'

In [32]:
adataprueba = adata[adata.obs["cell_type_coarse"].isin([cell_type[0]])]

In [34]:
adataprueba.obs

,sample,uicc_stage,sex,ever_smoker,driver_genes,condition,age,patient,tissue,origin,...,ROS_mutation,origin_fine,study,platform,platform_fine,cell_type_major,batch,_predictions,_leiden,_cell_type_tumor_predicted
AAACCTGCATTCTCAT-1_0-9,Lambrechts_Thienpont_2018_6653_BT1375,I,male,yes,NaN,LUSC,60.0,Lambrechts_Thienpont_2018_6653_7,lung,tumor_primary,...,NaN,tumor_primary,Lambrechts_Thienpont_2018,10x,10x_3p_v2,B cell,NaN,NaN,NaN,NaN
AAAGTAGCACGGTAGA-1_0-9,Lambrechts_Thienpont_2018_6653_BT1375,I,male,yes,NaN,LUSC,60.0,Lambrechts_Thienpont_2018_6653_7,lung,tumor_primary,...,NaN,tumor_primary,Lambrechts_Thienpont_2018,10x,10x_3p_v2,B cell,NaN,NaN,NaN,NaN
AACCGCGCAAACTGTC-1_0-9,Lambrechts_Thienpont_2018_6653_BT1375,I,male,yes,NaN,LUSC,60.0,Lambrechts_Thienpont_2018_6653_7,lung,tumor_primary,...,NaN,tumor_primary,Lambrechts_Thienpont_2018,10x,10x_3p_v2,B cell,NaN,NaN,NaN,NaN
AACGTTGCAGCTTCGG-1_0-9,Lambrechts_Thienpont_2018_6653_BT1375,I,male,yes,NaN,LUSC,60.0,Lambrechts_Thienpont_2018_6653_7,lung,tumor_primary,...,NaN,tumor_primary,Lambrechts_Thienpont_2018,10x,10x_3p_v2,B cell,NaN,NaN,NaN,NaN
ACACTGATCAGAAATG-1_0-9,Lambrechts_Thienpont_2018_6653_BT1375,I,male,yes,NaN,LUSC,60.0,Lambrechts_Thienpont_2018_6653_7,lung,tumor_primary,...,NaN,tumor_primary,Lambrechts_Thienpont_2018,10x,10x_3p_v2,B cell,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6506_5-18,UKIM-V_P3_tumor_primary,I,male,yes,NaN,LUAD,64.0,UKIM-V_P3,lung,tumor_primary,...,NaN,tumor_primary,UKIM-V,BD-Rhapsody,BD-Rhapsody,B cell,NaN,NaN,NaN,NaN
242982_5-18,UKIM-V_P3_tumor_primary,I,male,yes,NaN,LUAD,64.0,UKIM-V_P3,lung,tumor_primary,...,NaN,tumor_primary,UKIM-V,BD-Rhapsody,BD-Rhapsody,B cell,NaN,NaN,NaN,NaN
654973_5-18,UKIM-V_P3_tumor_primary,I,male,yes,NaN,LUAD,64.0,UKIM-V_P3,lung,tumor_primary,...,NaN,tumor_primary,UKIM-V,BD-Rhapsody,BD-Rhapsody,B cell,NaN,NaN,NaN,NaN
458636_5-18,UKIM-V_P3_tumor_primary,I,male,yes,NaN,LUAD,64.0,UKIM-V_P3,lung,tumor_primary,...,NaN,tumor_primary,UKIM-V,BD-Rhapsody,BD-Rhapsody,B cell,NaN,NaN,NaN,NaN


In [35]:
adata_dict = {}
for name in cell_type:
    name_ad = name.replace(" ","_")
    adata_name = f"{name_ad}_adata"
    adata_dict[adata_name] = adata[adata.obs["cell_type_coarse"].isin([name])]

In [10]:
# Compute distances in the PCA space, and find cell neighbors
sc.pp.neighbors(adata,use_rep="X_scANVI")

In [69]:
cell_type = list(adata.obs["cell_type_coarse"].unique())
deseq = "../../bin/deseq2.R"
deseq_results = "/data/projects/2023/atlas_protocol/results/differential_expression/deseq_resdir"
result_dir = "/data/projects/2023/atlas_protocol/results/differential_expression/cell_type"
cpus = 6

In [78]:
import pdb

In [74]:
#FOR TRIALS
cell_type = cell_type[0]

In [110]:
def run_deseq(count_table, sample_sheet, deseq_prefix, contrast, deseq_resdir):
    os.makedirs(deseq_resdir, exist_ok = True)
    pdb.set_trace()
   

    
    deseq_cmd = [deseq, count_table, sample_sheet,
                 "--cond_col", "condition",
                 "--c1", contrast[0],
                 "--c2", contrast[1], 
                 "--resDir", deseq_resdir, 
                 "--prefix", deseq_prefix, 
                 "--cpus", str(cpus), 
                 "--save_workspace"]
    
    stdout = open(deseq_resdir + "/" + deseq_prefix + ".log", 'w')
    stderr = open(deseq_resdir + "/" + deseq_prefix + ".err", 'w')
    sp.run(deseq_cmd, capture_output=False, stdout=stdout, stderr=stderr)
    stdout.close()
    stderr.close()

In [111]:
def save_pseudobulk(pb, samplesheet_filename, counts_filename):
    samplesheet = pb.obs.copy()
    samplesheet.reset_index(inplace=True)
    sample_ids_repl = fix_sample_ids(pb)
    bulk_df = pb.to_df().T.rename(columns=sample_ids_repl)
    bulk_df = pb.to_df().T
    bulk_df.index.name = "gene_id"
    samplesheet.to_csv(samplesheet_filename)
    bulk_df.to_csv(counts_filename)

In [112]:
def fix_sample_ids(pb):
    repl = {}
    for k,v in dict(zip(pb.obs["condition"].index, "_"+pb.obs["condition"].values)).items():
        repl[k] = k.replace(v,"")

    return(repl)

In [113]:
for ct ,tmp_ad in adata_dict.items():    
    pb = dc.get_pseudobulk(
        tmp_ad,
        sample_col='sample',
        groups_col='condition',
        layer='raw_counts',
        mode='sum',
        min_prop=0.05,
        min_cells=10,
        min_counts=1000,
        min_smpls=2
    )
    if pb.obs["condition"].nunique() <= 1:
        print(f"Cell type {ct} does not have enough replicates per group")
    else:
        contrast = ["LUSC", "LUAD"]
        contrast_str = f"{contrast[0]}_vs_{contrast[1]}"
        deseq_resdir = f"{deseq_results}/{contrast_str}"

        ct = ct.replace(" ", "_")
        deseq_prefix = f"{contrast_str}_{ct}"
        
        
        
        sample_sheet = f"{result_dir}/{deseq_prefix}.samplesheet.csv"
        count_table = f"{result_dir}/{deseq_prefix}.counts.csv"

        save_pseudobulk(pb, sample_sheet, count_table)
        run_deseq(count_table, sample_sheet, deseq_prefix, contrast, deseq_resdir)
    

> /tmp/ipykernel_12040/409787545.py(7)run_deseq()
      5 
      6 
----> 7     deseq_cmd = [deseq, count_table, sample_sheet,
      8                  "--cond_col", "condition",
      9                  "--c1", contrast[0],



ipdb>  deseq_prefix


'LUSC_vs_LUAD_B_cell_adata'


ipdb>  deseq_resdir


'/data/projects/2023/atlas_protocol/results/differential_expression/deseq_resdir/LUSC_vs_LUAD'


ipdb>  exit


In [ ]:
# Generate UMAP features
#sc.tl.umap(adata, init_pos = "X_umap")
# Visualize
#sc.pl.umap(adata, color=['cell_type'], frameon=False)

 stdout = open(deseq_resdir + "/" + deseq_prefix + ".log", 'w')

In [ ]:
for ct, tmp_ad in adata_dict:
    
    pb = dc.get_pseudobulk(
        tmp_ad,
        sample_col='patient',
        groups_col='cell_type_coarse',
        layer='raw_counts',
        mode='sum',
        min_prop=0.05,
        min_cells=10,
        min_counts=1000,
        min_smpls=2
    )
    if pb.obs["cell_type_coarse"].nunique() <= 1:
        print(f"Cell type {ct} does not have enough replicates per group")
    else:
        contrast = ["LUSC", "LUAD"]
        contrast_str = f"{contrast[0]}_vs_{contrast[1]}"
        deseq_resdir = f"{deseq_results}/{contrast_str}/"
        
        ct = ct.replace(" ", "_")
        deseq_prefix = f"{contrast_str}_{ct}"
        
        sample_sheet = f"{result_dir}/{deseq_prefix}.samplesheet.csv"
        count_table = f"{result_dir}/{deseq_prefix}.counts.csv"

        save_pseudobulk(pb, sample_sheet, count_table)
        run_deseq(count_table, sample_sheet, deseq_prefix, contrast, deseq_resdir)

In [ ]:
def run_deseq(count_table, sample_sheet, deseq_prefix, contrast, deseq_resdir):
    os.makedirs(deseq_resdir, exist_ok = True)
    
    deseq_cmd = [deseq, count_table, sample_sheet,
                 "--cond_col", "condition",
                 "--c1", contrast[0],
                 "--c2", contrast[1], 
                 "--resDir", deseq_resdir, 
                 "--prefix", deseq_prefix, 
                 "--cpus", str(cpus), 
                 "--save_workspace"]
    
    stdout = open(deseq_resdir + "/" + deseq_prefix + ".log", 'w')
    stderr = open(deseq_resdir + "/" + deseq_prefix + ".err", 'w')
    sp.run(deseq_cmd, capture_output=False, stdout=stdout, stderr=stderr)
    stdout.close()
    stderr.close()

In [ ]:
! ../../bin/deseq2.R /data/projects/2023/atlas_protocol/results/differential_expression/Tumorcells_samplesheet.csv /data/projects/2023/atlas_protocol/results/differential_expression/Tumorcells_counts.tsv --c1 "LUAD" --c2 "LUSC"

## OLD APPROACH 

In [ ]:
# Get pseudo-bulk profile
pdata = dc.get_pseudobulk(adata,
                          sample_col='patient',
                          groups_col='cell_type_coarse',
                          layer='raw_counts',
                          mode='sum',
                          min_cells=10,
                          min_counts=1000
                         )
pdata

In [ ]:
cell_type = padata.obs["cell_type_coarse"] 

In [ ]:
# Create counts 
patient = pdata_cell.obs["patient"] #patient id
gene_symbol = pdata_cell.var.index #gene id as index
counts_df  = pd.DataFrame(data = pdata_cell.X, columns = gene_symbol, index =patient) #counts dataframe
counts_df.index.name = None
counts_df = counts_df.T
counts_df.index.name = "gene_symbol"

resDir = '/data/projects/2023/atlas_protocol/results/differential_expression/'
cell_type_name_no_space = cell_type_name.replace(" ","")
filename_co = f"{cell_type_name_no_space}_counts.tsv"
file_path = os.path.join(resDir, filename_co)
counts_df.to_csv(file_path,sep = "\t",index = True)


In [ ]:
covariates = ['sex', 'ever_smoker', 'condition', 'age','tumor_stage', 'study', 'platform']
samplesheet_df = pdata_cell.obs.loc[:,covariates] # More columns can be added to be further used as covariates



samplesheet_df.index = samplesheet_df.index.str.replace("_"+cell_type_name,"")
samplesheet_df["sample"]= samplesheet_df.index

samplesheet_df.rename(columns = {"condition":"group"}, inplace = True) # Rename columns
samplesheet_df.index.name = None

filename_co = f"{cell_type_name_no_space}_samplesheet.csv"
file_path = os.path.join(resDir, filename_co)
samplesheet_df.to_csv(file_path,sep = ",",index = True)


In [ ]:
deseq = "../../bin/deseq2.R"
deseq_results = "/data/projects/2023/atlas_protocol/results/differential_expression/"